I made so simple model with LogisticRegressoion.
kernel of https://www.kaggle.com/tunguz helped me.
I'm beginner, so there may be many strange point. Please give me a advise.

------------------------------------------------------------------------------------------------

日本語で書かれたkernelsがほとんどなかったため
少しでも初心者の助けになればとこちらのkernelsを作成します。


ロジスティック回帰によるシンプルなモデルを構築しました。
https://www.kaggle.com/tunguz　のkernelを参考にさせていただきました。
初心者のため至らない点もあるかと思います。ぜひアドバイスお願いします。


In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import Ridge, LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings('ignore')

Reading data.

------------------------------------------------------------------------------------------------

データの読み込み。

In [ ]:
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

Observe shape of data.

-------------------------------------------------------------------------------------------------------

データの大きさを見てみる。

In [ ]:
print('Shape of train is ', train.shape)
print('shape of test is ',  test.shape)

Observe overview of data.

------------------------------------------------------

データの概要を見てみる。

In [ ]:
train.head()

In [ ]:
test.head()

Divide train data to 'target' and 'train'.
Setting 'id' column to index.

---------------------------------------------------------------------------------

学習用データを、"train"と”target"に分ける。
また、"id"をインデックスに指定する。

In [ ]:
target = train.iloc[:,[0,257]].set_index('id')
train = train.iloc[:,0:257].set_index('id')
test = test.set_index('id')

Observe distribution of target column.

↓

We can understand "Quantity of target 1 and 0 is almost same."

----------------------------------------------------------------------------------------------

"target" カラムの分布を観察する。

↓

１と０の数はほとんど同じである。


In [ ]:
print(target.describe())
plt.hist(target.values, bins=10)
plt.title('Histogram target counts')
plt.xlabel('Count')
plt.ylabel('Target')
plt.show()

Observe distribution of features column.

↓

We can understand "Most features have normal distribution."

-------------------------------------------------------------------------------

特徴量のカラムの分布を観察する。

↓

ほとんどすべての特徴量が正規分布を持つことがわかる。

In [ ]:
for n in range(2):
    plt.figure(figsize=(8, 3))
    plt.hist(train.iloc[:,n].values, bins=200)
    plt.title('Destribution of data' + str(n))
    plt.xlabel('Value')
    plt.ylabel('Count')
    plt.show()

But 'wheezy-copper-turtle-magic' has not normal distribution but like that.

----------------------------------------------------------------------------------------

しかし 'wheezy-copper-turtle-magic' カラムは少し特異な分布を示す。

In [ ]:
plt.figure(figsize=(8, 3))
plt.hist(train['wheezy-copper-turtle-magic'].values, bins=200)
plt.title('Histogram muggy-smalt-axolotl-pembus counts')
plt.xlabel('Value')
plt.ylabel('Count')
plt.show()

In [ ]:
train.describe()

In [ ]:
test.describe()

Normalize train and test data.

------------------------------------------------

学習用データとテスト用データを正規化する。

In [ ]:
scaler = StandardScaler()
scaler.fit(train)
train_scaled = pd.DataFrame(scaler.transform(train), columns=train.columns)
scaler.fit(test)
test_scaled = pd.DataFrame(scaler.transform(test), columns=test.columns)

Predict with LogisticRegression.
Validation with 5 split Kfold.
Print ROCAUC score.

----------------------------------------------------------------------------

ロジスティック回帰によって予測を行う。
五分割して交差検定を行う。
ROCAUCスコアを出力する。

In [ ]:
%%time
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
oof = np.zeros(train_scaled.shape[0])
pred = 0

for fold_, (train_index, validation_index) in enumerate(folds.split(train_scaled.values, target.values)):
    print("Fold {}".format(fold_+1))
    x_train, y_train = train_scaled.iloc[train_index], target.iloc[train_index]
    x_val, y_val = train_scaled.iloc[validation_index], target.iloc[validation_index]
    classifier = LogisticRegression(verbose=0, n_jobs=-1)
    classifier.fit(x_train, y_train)
    val_pred = classifier.predict_proba(x_val)[:,1]
    oof[validation_index] = val_pred
    pred += classifier.predict_proba(test_scaled)[:,1]/10
    print(roc_auc_score(y_val, val_pred))
    
print(roc_auc_score(target.values, oof))

Score is not good.
I made interactive model.

-----------------------------------------------

スコアはよくなかった。
相互作用項を考慮してモデルを構築してみる。

In [ ]:
from tqdm import tqdm_notebook as tqdm
from sklearn.svm import SVC

cols = [c for c in train.columns if c not in ['id']]
cols.remove('wheezy-copper-turtle-magic')
interactions = np.zeros((512,255))
oof = np.zeros(len(train))
preds = np.zeros(len(test))

for n in tqdm(range(512)):
    # ONLY TRAIN WITH DATA WHERE WHEEZY EQUALS I
    target2 = target[train['wheezy-copper-turtle-magic']==n]
    train2 = train[train['wheezy-copper-turtle-magic']==n]
    test2 = test[test['wheezy-copper-turtle-magic']==n]
    idx1 = []
    idx2 = []
    for m in range(262144):
        if train['wheezy-copper-turtle-magic'][m] == n:
            idx1.append(m)
    for l in range(131073):
        if test['wheezy-copper-turtle-magic'][l] == n:
            idx2.append(l)
    idx1 = np.array(idx1)
    idx2 = np.array(idx2)
    target2.reset_index(drop=True,inplace=True)
    train2.reset_index(drop=True,inplace=True)
    test2.reset_index(drop=True,inplace=True)

    
    folds = StratifiedKFold(n_splits=5, random_state=0)
    for train_index, test_index in folds.split(train2.values, target2.values):
        # LOGISTIC REGRESSION MODEL
        clf = SVC(probability=True,kernel='poly',degree=4,gamma='auto')
        clf.fit(train2.iloc[train_index],target2.iloc[train_index])
        oof[idx1[test_index]] = clf.predict_proba(train2.iloc[test_index])[:,1]
        preds[idx2] += clf.predict_proba(test2)[:,1] / 5.0
        # RECORD INTERACTIONS
#        for j in range(255):
#            if clf.coef_[0][j]>0: interactions[n,j] = 1
#            elif clf.coef_[0][j]<0: interactions[n,j] = -1
    #if i%25==0: print(i)

        
# PRINT CV AUC
auc = roc_auc_score(target,oof)
print('LR with interactions scores CV =',round(auc,5))

In [ ]:
auc = roc_auc_score(target,oof)
print('LR with interactions scores CV =',round(auc,5))

Making Submission csv.

--------------------------------------------------------

提出用データの作成。

In [ ]:
sample = pd.read_csv("../input/sample_submission.csv")
print(sample)
sample.target = preds
sample.to_csv('submisson.csv',index=False)